In [ ]:
from IPython.core.display import HTML
css = open('../data/style-table.css').read() + open('../data/style-notebook.css').read()
HTML('../data/<style>{}</style>'.format(css))

In [ ]:
%config IPCompleter.greedy = True

In [ ]:
import datetime, os


from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows

from journalfiles import JournalFiles
import pandas as pd
from structjour.pandasutil import DataFrameUtil, InputDataFrame, ToCSV_Ticket as Ticket
from structjour.tradeutil import ReqCol, FinReqCol, XLImage, TradeUtil
#TradeUtil, FinReqCol, ReqCol, 

jf=JournalFiles(theDate=datetime.date(2018, 9, 11), outdir = 'out/', mydevel=True)
jf._printValues()
        
tkt = Ticket(jf)

tu = TradeUtil()
trades, jf =tkt.newDFSingleTxPerTicket()
# trades = pd.read_csv(jf.inpathfile)

idf = InputDataFrame()
reqCol = ReqCol()
finalReqCol = FinReqCol()

DataFrameUtil.checkRequiredInputFields(trades, reqCol.columns)
trades = idf.zeroPadTimeStr(trades)
trades = trades.sort_values([reqCol.acct, reqCol.ticker, reqCol.time])
trades = idf.mkShortsNegative(trades)
swingTrade = idf.getOvernightTrades(trades)
swingTrade = idf.figureOvernightTransactions(trades)
trades = idf.insertOvernightRow(trades,swingTrade)

trades = tu.addFinReqCol(trades)
newTrades = trades[finalReqCol.columns]
newTrades.copy()
nt = newTrades.sort_values([finalReqCol.ticker,finalReqCol.acct,  finalReqCol.time])

In [ ]:
# nt.head()

In [ ]:
nt = tu.writeShareBalance(nt)
# nt.head()

In [ ]:
nt = tu.addStartTime(nt)
# nt.head()

In [ ]:
nt = nt.sort_values([finalReqCol.start, finalReqCol.acct, finalReqCol.time])
# nt.head()

In [ ]:
nt = tu.addTradeIndex(nt)
# nt.head()


In [ ]:
nt = tu.addTradePL(nt)
# nt.head()

In [ ]:
nt = tu.addTradeDuration(nt)
# nt.head()

In [ ]:
nt = tu.addTradeName(nt)
# nt.head()

In [ ]:
nt=DataFrameUtil.addRows(nt,1)
# nt.head()

In [ ]:
nt = tu.addSummaryPL(nt)
# nt.head()

In [ ]:
    
ldf=tu.getTradeList(nt)

In [ ]:
ldf[0]
df=ldf[0]
df

In [ ]:
from thetradeobject import SumReqFields

from thetradeobject import SumReqFields, TheTradeObject
import pandas as pd

srf = SumReqFields()

In [ ]:
tto = TheTradeObject(df, interview=True)

In [ ]:
# type(tto.TheTrade.Target)

In [ ]:
#  pd.to_numeric(tto.TheTrade.Target, errors='coerce')

In [ ]:
TheTrade = tto.TheTrade

In [ ]:
tto.runSummary()
df

In [ ]:
# TheTrade



In [ ]:
# TheTrade[['Entry1', 'Entry2', 'Entry3', 'Entry4', 'Entry5', ]]


### name

In [ ]:
ix = df.index[-1]
df.loc[ix]['Name']

In [ ]:
TheTrade[srf.name]=df.loc[ix]['Name']
TheTrade

### Account

In [ ]:
ix = df.index[-1]
df.loc[ix]['Account']

In [ ]:
TheTrade[srf.acct]="Live" if df.loc[ix]['Account'].startswith("U") else "SIM"
TheTrade

### Sum

In [ ]:
ix = df.index[-1]
df.loc[ix]['Sum']

In [ ]:
TheTrade[srf.PL]=df.loc[ix]['Sum']
TheTrade

### start

In [ ]:
ix = df.index[-1]
df.loc[ix]['Start']

In [ ]:
TheTrade[srf.start]=df.loc[ix]['Start']
TheTrade

### Duration

In [ ]:
df.loc[ix]['Duration']
time = df.loc[ix][srf.dur]
time

In [ ]:
duration = "{0} hours {1}:{2}".format(time.seconds // 3600, time.seconds // 60, time.seconds % 60)

TheTrade[srf.dur]=duration
TheTrade

### Strategy and note

In [ ]:
TheTrade.Strategy=""
strats = ["ORB", "ABCD", "VWAP Reversal", "Bull Flag", "Fallen ANgel",
      "VWAP False Breakout", "VWAP Reversal", "15 Minute Reversal",
      "VWAP MA trend", "Other", "Skip"]
def getStrategy() :


    s="What was the strategy?" 

    for i, strat in zip(range(1, len(strats)+1), strats) :
        s = "{0}\n     {1}. {2}".format(s, i, strat)

    s = s + "\n"
    #     print(i, strat)
    #     strategies = strategies + ""
    while True:
        try :
            reply = input(s)
            ireply = int(reply)
            if ireply < 1 or ireply > len(strats) :
                raise ValueError
            break
        except ValueError:
            print("Please enter a number between 1 and {0}".format(len(strats)))
            continue
    return ireply -1

reply = input ("Would you like to enter the strategy name?")
if reply.lower().startswith("y") :
    reply = getStrategy()
    

    if reply == 9 :
        response = input("What do you want to call the strategy?")
        TheTrade.Strategy = response
    elif reply == 10 :
        pass
    elif reply > -1 and reply < len(strats):
        TheTrade.Strategy=strats[reply]
    else :
        print("WTF?   ", reply)
        raise ValueError
TheTrade

###  Shares


In [ ]:
ix0 = df.index[0]
side = df.loc[ix0]['Side']

In [ ]:
side


In [ ]:
endSide=df.loc[ix]['Side']
endSide

In [ ]:
if side.startswith("B"):
    shares = df[srf.bal].max()
else :
    shares = df[frcsrf.bal].min()
    
TheTrade.Shares = "{0} shares".format(shares)
TheTrade

### Market Value

In [ ]:
if side.startswith("B"):
    shares = df[frc.bal].max()
else :
    shares = df[frc.bal].min()
shares * df[frc.price]

In [ ]:
 mkt = shares * df.loc[ix0][frc.price]
TheTrade['MktVal'] = mkt
TheTrade

### Entries

In [ ]:
df

In [ ]:
B=list()
S=list()

# side = "Long" if df.loc[ix0][frc.side].startswith('B') else None
# side

In [ ]:
for i, row in df.iterrows() :
    if(row[frc.side]).startswith('B') :
        B.append(row[frc.price])
    else :
        S.append(row[frc.price])
S

In [ ]:
TheTrade['EntriesHeader'] = "Entries"
if len(B) > 5 :
    more = len(B) - 5
    TheTrade.Entries5 =  "Plus {} more.".format(more)
for i, price in zip(range (len(B)), B) :
    col = "Entries" + str(i+1)
    TheTrade[col] = price
    
TheTrade
                    
                    

In [ ]:
TheTrade['ExitsHeader'] = "Exits"
if len(S) > 5 :
    more = len(S) - 5
    TheTrade.Entries5 =  "Plus {} more.".format(more)
for i, price in zip(range (len(S)), S) :
    col = "Exits" + str(i+1)
    TheTrade[col] = price
    
TheTrade

### Target Stop Description and Notes

In [ ]:
TheTrade.TargHead = "Target"
TheTrade.StopHead = "Stop"
TheTrade.RRHead = "R:R"
TheTrade.MaxHead = "Max Loss"
TheTrade[["Entries1", "Entries2", "Exits1", "Exits2"]]

In [ ]:
def enterTarget() :
    target = 0
    while True:
        try :
            targ = input("What was your target? ")
            if targ.lower().startswith('q') :
                target = 0
                break
            target = float(targ)
        except ValueError :
            print('''
            Please enter a number or 'q' to skip
            ''')
            target = 0
            
            continue
        break
    return target
target = enterTarget()
TheTrade.Target = target
TheTrade.TargDiff = target - TheTrade.Entries1

In [ ]:

TheTrade[['Entries1', 'Target', 'TargDiff', 'StopLoss' , 'SLDiff', 'MaxLoss']]

In [ ]:
def enterStop() :
    stop = 0
    while True:
        try :
            stop = input("What was your Stop Loss? ")
            if stop.lower().startswith('q') :
                stop = 0
                break
            stop = float(stop)
        except ValueError :
            print('''
            Please enter a number or 'q' to skip
            ''')
            continue
        break
    return stop
stop = enterStop()
TheTrade.StopLoss = stop
TheTrade.SLDiff = stop - TheTrade.Entries1

### Consider placing this bit of code with the code to get the shares data or saving the shares data seperately as a float as we also need the stop loss diff (SLDiff)

In [ ]:
x=TheTrade.Shares.str.split()
x = float(x[0][0])
maxloss = TheTrade.SLDiff * x
maxloss

In [ ]:

TheTrade.MaxLoss = stop
TheTrade.RR = stop / target
TheTrade[['Entries1', 'Target', 'TargDiff', 'StopLoss' , 'SLDiff', 'MaxLoss']]

In [ ]:
TheTrade[['Entries1', 'Target', 'TargDiff', 'StopLoss' , 'SLDiff', 'RR', 'MaxLoss']]


### Note that the interview stuff will be taken care of during the image insertion allowing the user to do all the review of a tradeafter viewing the chart. The interview stuff will include: 1. strategy  2. brief strategy note  3. target  4. stop loss  5. description of trade.  6. Notes and analysis of the trade

### On Second thought, the strategy note, trade explanation and analysis will be left to do in excel. The Gui Version os structjour will implement the review. Otherwise I would have to include some facility for entering multiple sentences in the console app.  Not that difficult but not that helpful when it can be done in Excel wilh all the visuals present.

In [ ]:
s="0"
f=float(s)
f